In [2]:
import pandas as pd
import requests
import os
import cv2
import json
import numpy as np

vote_count_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv'
vote_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] Votelog.csv'

In [3]:
# images_p = [os.path.join('test/pdf/20190823113634A11', f) for f in os.listdir('test/pdf/20190823113634A11')]
# images = [cv2.imread(f) for f in images_p]

In [4]:
def parse_text(image):
      gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
      blured = cv2.GaussianBlur(gray, (1, 9), 0)
      th, threshed = cv2.threshold(blured, 200, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
      threshed = cv2.dilate(threshed, kernel)
      hist = cv2.reduce(threshed, 1, cv2.REDUCE_AVG).reshape(-1)

      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
      dilated = cv2.dilate(threshed, kernel)
      contours, hier  = cv2.findContours(
            dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      rects = [cv2.boundingRect(c) for c in contours]
      rects.sort(key=lambda rect: rect[1])

      return rects


In [5]:
os.path.abspath('')

'/Users/napatswift/wevis/ocr-vote-log'

In [6]:
pdf_dir = 'test/pdf'
images = {}
for dirpath, dir, filenames in os.walk(pdf_dir):
  # if dirpath == pdf_dir: continue
  for fname in filenames:
    if not fname.endswith('.png'): continue
    fpath = os.path.join(dirpath, fname)
    image = cv2.imread(fpath)
    images[fpath] = image



In [6]:
len(images.keys())

1232

In [7]:
image.shape

(2088, 1480, 3)

In [23]:
pred = []
image_boxes = []
for fpath, image in images.items():
  boxes = parse_text(image)
  image_boxes.append(boxes)
      

In [ ]:
def label_studio_input(boxes, fpath):
  for x, y, width, height in boxes:
    size.append(width*height)
    ratio.append(width/height)
    if width * height < 250 or width/height < 0.2: continue
    results.append({
          "original_width": imw,
          "original_height": imh,
          "image_rotation": 0,
          "value": {
            "x": (x/imw)*100,
            "y": (y)/imh*100,
            "width":width/imw*100,
            "height": height/imh*100,
            "rotation": 0,
            "rectanglelabels": [
              "text"
            ]
          },
          "from_name": "label",
          "to_name": "image",
          "type": "rectanglelabels",
        },)
    imabspath = os.path.abspath(fpath)
    imid += 1
    return {
      'data': {'ocr': '/data/local-files/?d='+imabspath},
      'predictions': [{
        'model_version': '0.0.1',
        'id': id,
        'result': results,
      }]
    }

In [16]:
with open('ds/pred.json', 'w') as f:
  json.dump(pred, f, indent=3)

# validate

In [15]:
import numpy as np

In [13]:
size = []
for impred in pred:
  for prediction in impred['predictions']:
    new_res = []
    for result in prediction['result']:
      w = result['value']['width']
      h = result['value']['height']
      size.append(w*h)
      if w*h > 1:
        new_res.append(result)
    # annot['result'] = list()
    # annot['predictions'] = new_res

# Column

In [18]:
def detect_column(images):
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 5))
  hist_avg = None
  for im in images:
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    th, threshed = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(threshed, kernel, iterations=3)

    if hist_avg is None:
      hist_avg = cv2.reduce(dilated, 0, cv2.REDUCE_AVG)
    else:
      hist_avg = (hist_avg + cv2.reduce(dilated, 0, cv2.REDUCE_AVG)) / 2

  hist_avg = hist_avg.reshape(-1)

  lined = im.copy()
  hist_b = hist_avg < hist_avg.max() - hist_avg.std()
  cols1 = np.argwhere(hist_b[1:] & (hist_b[1:] ^ hist_b[:-1])).reshape(-1)
  cols2 = np.argwhere(hist_b[:-1] & (hist_b[1:] ^ hist_b[:-1])).reshape(-1)
  cols = (cols1 + ((cols1 - cols2) * 0.8)).astype(int)

  return cols

In [19]:
sample = [
  im for i, im in
enumerate(images.values()) if i < 5]

In [20]:
cols = detect_column(sample[:1])

In [244]:
def cell_span(boxes):
  x0 = boxes[:, 0]
  x1 = (x0 + boxes[:, 2]).max()
  x0 = x0.min()
  y0 = boxes[:, 1]
  y1 = (y0 + boxes[:, 3]).max()
  y0 = y0.min()
  return (x0, y0, x1-x0, y1-y0)


In [249]:
def draw_boxes(boxes, image, self):
  canvas = image.copy()
  for x, y, w, h in boxes:
    continue
    canvas = cv2.rectangle(canvas, (x, y), (x+w, y+h), 255, 1)
  x, y, w, h = self
  canvas = cv2.rectangle(canvas, (x, y), (x+w, y+h), 1, 2)
  x0, y0, w, h = cell_span(boxes)
  canvas = cv2.rectangle(canvas, (x0, y0), (x0+w, y0+h), (100, 100, 1000), 1)
  return canvas

In [250]:
for image, boxes in zip(images.values(), image_boxes):
  boxes = np.array(boxes)
  break

In [252]:
def line():
  # create -1 matrix
  box_line_num = np.zeros(len(boxes))
  box_line_num.fill(-1)
  # sorting by x axis
  boxes = np.array(sorted(boxes, key=lambda x: x[0]))
  i = 0
  line_image = image.copy()
  while (box_line_num == -1).any():
    if box_line_num[i] != -1:
      i += 1
      continue
    center = boxes[:, 1] + (boxes[:, 3]/2)
    angle = np.arctan2((center - center[i]), (boxes[:, 0] - boxes[i, 0]))

    is_in_line = (np.abs(angle) < 0.03)
    is_in_line_index = np.argwhere(is_in_line).reshape(-1)

    line_boxes = boxes[is_in_line_index]
    box_line_num[is_in_line] = i

    # line_image = draw_boxes(line_boxes, line_image, boxes[i])
    # cell_spans.append(cell_span(line_boxes))
    i += 1
  
  return box_line_num

True

In [255]:
cell_spans = np.array(cell_spans)

In [260]:
cell_spans[:, 2]

array([1177, 1077, 1077, 1076, 1077, 1077, 1077, 1077, 1076, 1077, 1077,
       1076, 1076, 1077, 1077, 1077, 1077, 1077, 1077, 1077, 1077, 1077,
       1077, 1077, 1077, 1078, 1075, 1075, 1076, 1076, 1076, 1076, 1074,
       1074,  606,  325,  145])